In [7]:
import os
import sys

# Add project root to PYTHONPATH automatically
PROJECT_ROOT = r"C:\Users\adib4\OneDrive\Documents\Projets perso\CongestionAI\find_issues.ipynb"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from src.model_pipelines.dl_pipeline import train_model, evaluate, predict
from src.utils.model_evaluation import evaluate_and_plot_block
from src.utils.hist_baseline import historical_baseline_multi
from src.utils.preprocessing import cyclical_encode, scale_features, encode_detectors
from src.utils.sequences import create_nhits_sequences, NHitsDataset
from src.utils.plots import plot_training_curves
from src.model_pipelines.losses import (
    SpikeWeightedMSELoss,
    TwoTermSpikeLoss,
    DeltaLoss,
    LossConfig,
    create_loss
)

from src.utils.crafted_features import (
    SpikeFeatureConfig,
    add_spike_features,
    add_lags_and_drop
)



from src.models.n_hits import NHitsForecaster
from src.models.tcn_forecaster import MultiHeadTCNForecaster

FILE_PATH = "prepared_data/preprocessed_full_data.csv"

In [2]:
def run_dl_experiment(
    model,
    optimizer,
    criterion,
    X_train_hist,
    Y_train,
    train_det_idx,
    X_val_hist,
    Y_val,
    val_det_idx,
    X_test_hist,
    Y_test,
    test_det_idx,
    device="cuda",
    batch_size=128,
    epochs=10,
    grad_clip=1.0,
    scheduler=None,
    scaler=None,
    exp_name="",
    patience=None,
):
    """
    Runs the full deep-learning training pipeline.

    - Builds model + dataloaders
    - Trains the model
    - Returns model, predictions, losses
    """

    # -------------------------
    # DATALOADERS
    # -------------------------
    train_loader = DataLoader(
        NHitsDataset(X_train_hist, Y_train, train_det_idx),
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        pin_memory=True
    )

    val_loader = DataLoader(
        NHitsDataset(X_val_hist, Y_val, val_det_idx),
        batch_size=batch_size,
        shuffle=False,
        pin_memory=True
    )

    if X_test_hist is None or Y_test is None or test_det_idx is None:
        test_loader = None
    else:
        test_loader = DataLoader(
            NHitsDataset(X_test_hist, Y_test, test_det_idx),
            batch_size=batch_size,
            shuffle=False,
            pin_memory=True
        )

    model.to(device)

    # -------------------------
    # TRAINING
    # -------------------------
    train_losses, val_losses, best_state = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        scaler=scaler,
        device=device,
        num_epochs=epochs,
        grad_clip=grad_clip,
        patience=patience,
    )

    # Save losses to file
    os.makedirs(f"plots_training_dl/{broad_exp_name}/", exist_ok=True)
    with open(f"plots_training_dl/{broad_exp_name}/losses_{exp_name}.txt", "w") as f:
        f.write("epoch,train_loss,val_loss\n")
        for i, (t_loss, v_loss) in enumerate(zip(train_losses, val_losses)):
            f.write(f"{i+1},{t_loss:.6f},{v_loss:.6f}\n")

    plot_training_curves(train_losses, val_losses, filename=f"training_curve{exp_name}.png", dir = f"plots_training_dl/{broad_exp_name}/")
    #model.load_state_dict(best_state)
    if test_loader is not None:
        _, test_loss = evaluate(model, test_loader, criterion, device)
        print(f"Test Loss ({exp_name}): {test_loss:.4f}")

    return model, train_losses, val_losses

In [3]:
def prepare_eval_df(df, idx_seq, preds, horizon):
    """
    df: the dataset (train, val or test)
    idx_seq: array of starting indices returned by create_nhits_sequences
    preds: model predictions (N, horizon)
    """

    df_subset = df.loc[idx_seq].copy()
    print(df.info())
    print(df_subset.info())
    print(len(df_subset), len(idx_seq), preds.shape)

    eval_df = pd.DataFrame({
        "row_idx": idx_seq,
        "timestamp": df_subset["timestamp"].values,
        "detector_id": df_subset["detector_id"].values,
    })
    

    # Add predictions
    for h in range(1, horizon + 1):
        eval_df[f"pred_{h}h"] = preds[:, h-1].numpy()

    # Add ground truth targets
    for h in range(1, horizon + 1):
        eval_df[f"future_{h}h"] = (
            df.groupby("detector_id")["congestion_index"]
              .shift(-h)
              .loc[idx_seq]
              .values
        )

    return eval_df.dropna()

In [4]:
def prepare_dl_data_with_spikes(history_offsets, forecast_horizon, nb_detectors, df_base,
                                years_split, feature_cols_norm, feature_cols_base,
                                weather_lags, spike_config=None):
    """Extended data prep with optional spike features."""
    
    print("Loading data...")
    df_small = df_base[df_base["detector_id"].isin(df_base["detector_id"].unique()[:nb_detectors])].copy()
    df_small = df_small.sort_values(["detector_id", "timestamp"])
    
    # Season encoding
    df_small.loc[(df_small["month"] <= 2) | (df_small["month"] == 12), "season"] = 0
    df_small.loc[(df_small["month"] > 2) & (df_small["month"] <= 5), "season"] = 1
    df_small.loc[(df_small["month"] > 5) & (df_small["month"] <= 8), "season"] = 2
    df_small.loc[(df_small["month"] > 8) & (df_small["month"] <= 11), "season"] = 3
    
    # Add spike features if configured
    feature_cols = feature_cols_base.copy()
    feature_cols_norm_full = feature_cols_norm.copy()
    
    if spike_config is not None:
        print(f"Adding spike features: deltas={spike_config.enable_deltas}, rolling={spike_config.enable_rolling_stats}")
        df_small = add_spike_features(df_small, spike_config)
        spike_feature_cols = spike_config.get_feature_columns()
        spike_norm_cols = spike_config.get_normalization_columns()
        feature_cols = feature_cols + spike_feature_cols
        feature_cols_norm_full = feature_cols_norm_full + spike_norm_cols
        print(f"  Added columns: {spike_feature_cols}")
    
    # Detector encoding
    df_small, det2idx = encode_detectors(df_small)
    
    # Add weather lag column names
    if "temperature" in feature_cols:
        feature_cols = feature_cols + [f"temperature_lag_{lag}h" for lag in weather_lags] \
            + [f"precipitation_lag_{lag}h" for lag in weather_lags] \
            + [f"visibility_lag_{lag}h" for lag in weather_lags]
    
    # Split
    train = df_small[df_small["timestamp"].dt.year.isin(years_split[0])].copy()
    val = df_small[df_small["timestamp"].dt.year.isin(years_split[1])].copy()
    test = df_small[df_small["timestamp"].dt.year.isin(years_split[2])].copy() if years_split[2] else None
    
    train = train.set_index("orig_idx")
    val = val.set_index("orig_idx")
    if test is not None:
        test = test.set_index("orig_idx")
    
    # Normalization
    minmax_cols = ["lon", "lat", "year", "season"]
    train, val, test, std_scaler, mm_scaler = scale_features(
        train, val, test, feature_cols_norm_full, latlon_cols=minmax_cols
    )
    
    # Weather lags
    if "temperature" in feature_cols_base:
        train = add_lags_and_drop(train, weather_lags)
        val = add_lags_and_drop(val, weather_lags)
        if test is not None:
            test = add_lags_and_drop(test, weather_lags)
    
    # Drop NaNs from spike features
    if spike_config is not None:
        spike_cols_in_df = [c for c in spike_feature_cols if c in train.columns]
        train = train.dropna(subset=spike_cols_in_df)
        val = val.dropna(subset=spike_cols_in_df)
        if test is not None:
            test = test.dropna(subset=spike_cols_in_df)
    
    # Keep only needed columns (congestion_index is already in feature_cols)
    keep_cols = feature_cols + ["timestamp", "detector_id", "det_index"]
    keep_cols = [c for c in keep_cols if c in train.columns]
    
    train = train[keep_cols]
    val = val[keep_cols]
    if test is not None:
        test = test[keep_cols]
    
    # Build sequences
    X_train_hist, Y_train, idx_train, det_train = create_nhits_sequences(
        train, feature_cols, history_offsets, forecast_horizon)
    X_val_hist, Y_val, idx_val, det_val = create_nhits_sequences(
        val, feature_cols, history_offsets, forecast_horizon)
    
    if test is not None:
        X_test_hist, Y_test, idx_test, det_test = create_nhits_sequences(
            test, feature_cols, history_offsets, forecast_horizon)
    else:
        X_test_hist, Y_test, idx_test, det_test = None, None, None, None
    
    print(f"Sequences created. Features: {len(feature_cols)}, Train samples: {len(Y_train)}")
    
    return (X_train_hist, Y_train, idx_train, det_train,
            X_val_hist, Y_val, idx_val, det_val,
            X_test_hist, Y_test, idx_test, det_test,
            train, val, test, std_scaler, mm_scaler)

In [17]:
def main(nb_detectors, forecast_horizon, history_offsets, exp_name, 
                   df_base, feature_cols_norm, feature_cols, weather_lags, model_config=None,
                   years_split=([2019,2020,2021,2022,2023,2024], [2018], [2016]),
                   evaluation_years=None, spike_config=None,
                   spike_eval_threshold=0.38, criterion=None):
    dir = f"plots_training_dl/{broad_exp_name}/"

    X_train_hist, Y_train, idx_train, det_train, \
    X_val_hist, Y_val, idx_val, det_val, \
    X_test_hist, Y_test, idx_test, det_test, \
    train, val, test, \
    std_scaler, mm_scaler = prepare_dl_data_with_spikes(history_offsets, 
                                            forecast_horizon, 
                                            nb_detectors, df_base,
                                            feature_cols_norm=feature_cols_norm, 
                                            feature_cols_base=feature_cols, 
                                            weather_lags=weather_lags,
                                            years_split=years_split,
                                            spike_config=spike_config)
    
    if criterion is None:
        criterion = nn.MSELoss()
    model = MultiHeadTCNForecaster(**model_config, num_features=X_train_hist.shape[-1])    
    params_experiment = {
        "model": model,
        "optimizer": torch.optim.Adam(model.parameters(), lr=1e-4),
        "criterion": criterion,
        "X_train_hist": X_train_hist,
        "Y_train": Y_train,
        "train_det_idx": det_train,
        "X_val_hist": X_val_hist,
        "Y_val": Y_val,
        "val_det_idx": det_val,
        "X_test_hist": X_test_hist,
        "Y_test": Y_test,
        "test_det_idx": det_test,
        "device": "cuda",
        "batch_size": 512,
        "epochs": 2,
        "grad_clip": None,
        "scheduler": None
    }
        
    if evaluation_years is None:
        evaluation_years = years_split[1]
    
    # RUN EXPERIMENT
    model, train_losses, val_losses = run_dl_experiment(**params_experiment, exp_name=exp_name)
    eval_df = prepare_eval_df(val, idx_val, predict(model, X_val_hist, det_val), forecast_horizon)
    eval_df["congestion_index"] = val.loc[idx_val, "congestion_index"].values
    evaluate_and_plot_block(eval_df, horizon=forecast_horizon, years=evaluation_years, plot_years=evaluation_years, 
                            filename=exp_name,
                            dir=dir, max_blocks=15,
                            eval_spikes=True,
                            spike_threshold=spike_eval_threshold)


In [6]:
df_base = pd.read_csv(FILE_PATH)
df_base["timestamp"] = pd.to_datetime(df_base["timestamp"])
df_base["orig_idx"] = df_base.index
df_base = cyclical_encode(df_base)

In [ ]:


# Fixed parameters
feature_cols_norm_base = [
    "temperature", "precipitation", "visibility", "congestion_index", "free_flow_speed"
]
feature_cols_base = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "lon", "lat", "year", "season",
    "temperature", "precipitation", "visibility",
    "congestion_index", "free_flow_speed"
]
nb_detectors = 20
years_split = [[2016, 2017, 2018, 2020, 2021, 2022, 2023, 2024], [2019], []]

# Fixed for now
forecast_horizon = 8
h_offsets = list(range(24))
w_lags = [-1, -2, -6, -8]
spike_trigger_threshold = 0.15
eval_spike_threshold = 0.38
spike_config = SpikeFeatureConfig(
        enable_deltas=True,
        enable_abs_deltas=False,
        enable_rolling_stats=False,
        delta_lags=[1, 2, 4, 6],
)
cfg_loss = LossConfig(loss_type="spike_weighted", spike_weight=3.0, spike_threshold=spike_trigger_threshold)
criterion = create_loss(cfg_loss)

broad_exp_name = "multi_head_TCN-best_cfg_gridsearch_spike_features"
exp_name = "test_pipeline"

model_config = {"horizon": forecast_horizon,
                "num_detectors": nb_detectors,
                "emb_dim": 256,
                "num_channels": (128, 256, 256),
                "kernel_size": 3,
                "dropout": 0.1,
                "use_se": False,
                "pooling": "last"
            }


main(nb_detectors=nb_detectors,
     forecast_horizon=forecast_horizon,
     history_offsets=h_offsets,
     exp_name=exp_name,
     df_base=df_base,
     feature_cols_norm=feature_cols_norm_base,
     feature_cols=feature_cols_base,
     weather_lags=w_lags,
     model_config=model_config,
     years_split=years_split,
     evaluation_years=[2019],
     spike_config=spike_config,
     spike_eval_threshold=eval_spike_threshold,
     criterion=criterion)

Loading data...
Adding spike features: deltas=True, rolling=False
  Added columns: ['delta_1h', 'delta_2h', 'delta_4h', 'delta_6h']
Sequences created. Features: 31, Train samples: 1068551


Epoch 1/2 - training:   0%|          | 0/2087 [00:00<?, ?it/s]


TypeError: 'LossConfig' object is not callable